Cargando los datos desde el fichero JSON con funciones para cada caso.


In [ ]:
import json
js = json.load(open('data-fill.json','r'))

def get_panam_info(country,medal,org=0):
    x = []
    y = []
    m = js['countries'][country]['panam']
    for i in range(4,18):
        if js['countries'][country]['panam'][str(i)]!=None and js['countries'][country]['panam'][str(i-1)]!=None:
            t = [i]+[m[str(i)][3]]+m[str(i-1)]+[i-1]+m[str(i-2)]+[i-2]+m[str(i-3)] + [i-3]
            x.append(t)
            if medal=='oro':
                y.append(m[str(i)][0])
            elif medal=='plata':
                y.append(m[str(i)][1])
            if medal=='bronce':
                y.append(m[str(i)][2])
    p = [18]+[org]+ m['17']+[17]+m['16']+[16]+m['15'] + [15]
    #print p
    return x,y,p

def get_olympic_info(country,medal,org=0):
    x = []
    y = []
    p = js['pairings']
    o = js['countries'][country]['olympic']
    m = js['countries'][country]['panam']
    for i in range(4,18):
        if js['countries'][country]['panam'][str(i)]!=None and js['countries'][country]['panam'][str(i-1)]!=None:
            to = o[str(p[str(i)][1])]+[int(p[str(i)][1])]+o[str(p[str(i-1)][1])]+[int(p[str(i-1)][1])]+o[str(p[str(i-2)][1])]+[int(p[str(i-2)][1])]
            t = [i]+[m[str(i)][3]]+m[str(i-1)]+[i-1]+m[str(i-2)]+[i-2]+m[str(i-3)]+[i-3]+to
            x.append(t)
            if medal=='oro':
                y.append(m[str(i)][0])
            elif medal=='plata':
                y.append(m[str(i)][1])
            if medal=='bronce':
                y.append(m[str(i)][2])
    p = [18]+[org]+ m['17']+[17]+m['16']+[16]+m['15']+[15]+o['28']+[28]+o['27']+[27]+o['26']+[26]
    return x,y,p

def get_centroam_info(country,medal,org=0):
    x = []
    y = []
    p = js['pairings']
    o = js['countries'][country]['olympic']
    m = js['countries'][country]['panam']
    c = js['countries'][country]['centro']
    for i in range(4,18):
        if js['countries'][country]['panam'][str(i)]!=None and js['countries'][country]['panam'][str(i-1)]!=None:
            tc = c[str(p[str(i)][2])]+[int(p[str(i)][2])]+c[str(p[str(i-1)][2])]+[int(p[str(i-1)][2])]+c[str(p[str(i-2)][2])]+[int(p[str(i-2)][2])]
            to = o[str(p[str(i)][1])]+[int(p[str(i)][1])]+o[str(p[str(i-1)][1])]+[int(p[str(i-1)][1])]+o[str(p[str(i-2)][1])]+[int(p[str(i-2)][1])]
            t = [i]+[m[str(i)][3]]+m[str(i-1)]+[i-1]+m[str(i-2)]+[i-2]+m[str(i-3)]+[i-3]+to+tc
            x.append(t)
            if medal=='oro':
                y.append(m[str(i)][0])
            elif medal=='plata':
                y.append(m[str(i)][1])
            if medal=='bronce':
                y.append(m[str(i)][2])
    p = [18]+[org]+ m['17']+[17]+m['16']+[16]+m['15']+[15]+o['28']+[28]+o['27']+[27]+o['26']+[26]+c['23']+[23]+c['22']+[22]+c['21']+[21]
    return x,y,p


Definiendo la función para predecir la cantidad de medallas. Recibe como parámetros el país, el tipo de medalla, la función para cargar los dato y si el país es organizador de los juegos o no.

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error


def predict_medal(country,medal,fdata,org=0):
    d = fdata(country, medal)
    
    gb = GradientBoostingRegressor(loss="huber")
    gb.fit(d[0],d[1])
    gb_all = gb.predict(d[0])
    gb_pre = gb.predict(d[2])
    gb_arr = np.append(gb_all,gb_pre)
    gb_err_abs = mean_absolute_error(gb_all,d[1])
    plt.plot([i[0] for i in d[0]]+[18],gb_arr, color="blue",label="gb")
    
    en = ElasticNet()
    en.fit(d[0],d[1])
    en_all = en.predict(d[0])
    en_pre = en.predict(d[2])
    en_arr = np.append(en_all,en_pre)
    en_err_abs = mean_absolute_error(en_all,d[1])
    plt.plot([i[0] for i in d[0]]+[18],en_arr, color="yellow",label="en")
    
    
    m_all = [(gb_all[i]+en_all[i])/2 for i in range(0,len(gb_all))]
    m_pre = [(gb_pre[-1]+en_pre[-1])/2]
    m_arr = np.append(m_all,m_pre)
    m_err_abs = mean_absolute_error(m_all,d[1])
    plt.plot([i[0] for i in d[0]]+[18],m_arr, color="green",label="media")
    
    plt.scatter([i[0] for i in d[0]],d[1], color = 'red', label="real")
    plt.legend(loc="upper left")
    plt.xlabel(u'Edición')
    plt.ylabel(u'Medallas de '+medal)
    plt.savefig(country+'-'+fdata.func_name+'-'+medal+'.png')
    plt.close()
    
    return m_pre[0], m_err_abs

Predcir los pronósticos para cada país por cada tipo de medalla. En cada caso se genera la imagen de la curva de predicción.

In [ ]:
pro = {}
medals = ['oro','plata','bronce']

for c in js['countries']:
    pro[c]={'panam':{'oro':{'cant':0,'error':0},'plata':{'cant':0,'error':0},'bronce':{'cant':0,'error':0}},'olympic':{'oro':{'cant':0,'error':0},'plata':{'cant':0,'error':0},'bronce':{'cant':0,'error':0}}}

    
for c in js['countries']:
    for m in medals:
        if c!='pe':
            pp = predict_medal(c,m,get_panam_info,org=0)
            po = predict_medal(c,m,get_olympic_info,org=0)
            pro[c]['panam'][m]['cant']=pp[0]
            pro[c]['panam'][m]['error']=pp[1]                               
            pro[c]['olympic'][m]['cant']=po[0]
            pro[c]['olympic'][m]['error']=po[1]
        else:
            pp = predict_medal(c,m,get_panam_info,org=0)
            po = predict_medal(c,m,get_olympic_info,org=0)
            pro[c]['panam'][m]['cant']=pp[0]
            pro[c]['panam'][m]['error']=pp[1]                               
            pro[c]['olympic'][m]['cant']=po[0]
            pro[c]['olympic'][m]['error']=po[1]

Salvar los datos de predicción en un fuchero JSON

In [ ]:
json.dump(pro,open('prediction.json','w'))